In [2]:
import numpy as np
import scipy.special as sc
import matplotlib.pyplot as plt
from numpy import random
from scipy.integrate import quad, trapezoid, fixed_quad

In [3]:
a = 1.2
b = 3.1
mu = 0.9426
N = 1/sc.beta(a+1,b+1) #third parameter
Pz = 5*2*np.pi / 48.0
constant = 0.4044*4/(3*2*np.pi)
delta = 10**-6
#upper_bound=100
upper_bound = 10

In [4]:
x = np.linspace(0, 1, 1000)
value = np.zeros(len(x))

# Actual plot is of x*q(x)
for i in range(0,len(x)):
    value[i] = N * x[i]**(a+1) * (1-x[i])**b
#plt.plot(x,value)
#plt.title("Transversity")
#plt.xlabel("x")
#plt.ylabel("x*q(x)")
#plt.show()

In [9]:
z = np.arange(0,9)
x_range = np.arange(10**-6, 1, 10**-6)
#print(z)
#print(x_range)

In [10]:
# ME real
A_r_LO = np.zeros(len(z))

# 2D array where each individual array is for the different z values. Each input in each array is for a differnt x value
A1_r = np.zeros((len(z),len(x_range)))
A2_r = np.zeros((len(z),len(x_range)))
A3_r = np.zeros((len(z),len(x_range)))
A4_r = np.zeros((len(z),len(x_range)))
A5_r = np.zeros((len(z),len(x_range)))
A6_r = np.zeros((len(z),len(x_range)))

A_r_NLO = np.zeros((len(z),len(x_range)))
A_r1_trap = np.zeros(len(z))
A_r2_trap = np.zeros(len(z))
A_r3_trap = np.zeros(len(z))
A_r4_trap = np.zeros(len(z))
A_r5_trap = np.zeros(len(z))
A_r6_trap = np.zeros(len(z))
A_r = np.zeros(len(z))

ME_r = np.zeros(len(z))

#LO
A_r_LO_func = lambda z: quad(lambda x: np.cos(x*Pz*z) * (N * x**a * (1-x)**b),0,1)[0]

#NLO
# Theta function implemented by changing integration bound to depend on x*(1-delta)
#n=100000
rA1_func = lambda x,z: (1/x)*constant * fixed_quad(lambda y:np.cos(y*Pz*z) * ((2*y/(x-y))*(np.log((x*Pz/mu)**2) + np.log(4*(y/x)*(1-(y/x)))) - (2*y/(x-y))), 0, x*(1-delta), n=10)[0] * (N * x**a * (1-x)**b)
rA2_func = lambda x,z: np.cos(x*Pz*z) * constant * fixed_quad(lambda xi: ((2*xi/(1-xi))*(np.log((x*Pz/mu)**2) + np.log(4*xi*(1-xi))) - (2*xi/(1-xi))), x, 1-delta, n=10)[0] * (N * x**a * (1-x)**b)
rA3_func = lambda x,z: (1/x)* fixed_quad(lambda y: np.cos(y*Pz*z)*constant*(2*(y/x)/(1-(y/x)) *np.log((y/x)/((y/x)-1)) + 2/(y/x)), x*(1+delta), upper_bound, n=10)[0] * (N * x**a * (1-x)**b)
rA4_func = lambda x,z: np.cos(x*Pz*z)*constant*fixed_quad(lambda xi: ((2*xi/(1-xi))*np.log(xi/(xi-1)) + (2/xi)), 1+delta, upper_bound,n=10)[0]* (N * x**a * (1-x)**b)
rA5_func = lambda x,z: (-1/x)*constant* fixed_quad(lambda y: np.cos(y*Pz*z) * ((2*y/(x+y))*np.log(y/(x+y)) + 2/(1+(y/x))), 0, upper_bound,n=10)[0] * (N * x**a * (1-x)**b)
rA6_func = lambda x,z: np.cos(x*Pz*z)*constant * fixed_quad(lambda xi:((-2*xi/(1-xi))*np.log(xi/(xi-1)) + (2/(1-xi))), -1*upper_bound, -x, n=10)[0] * (N * x**a * (1-x)**b)   

for i in range(0,len(z)):
    A_r_LO[i] = A_r_LO_func(z[i])
    for j in range(0,len(x_range)):
        A1_r[i][j] = np.array(rA1_func(x_range[j], z[i]))
        A2_r[i][j] = np.array(rA2_func(x_range[j], z[i]))
        A3_r[i][j] = np.array(rA3_func(x_range[j], z[i]))
        A4_r[i][j] = np.array(rA4_func(x_range[j], z[i]))
        A5_r[i][j] = np.array(rA5_func(x_range[j], z[i]))
        A6_r[i][j] = np.array(rA6_func(x_range[j], z[i]))
        
    A_r1_trap[i] = trapezoid(A1_r[i],x_range)
    A_r2_trap[i] = trapezoid(A2_r[i],x_range)
    A_r3_trap[i] = trapezoid(A3_r[i],x_range)
    A_r4_trap[i] = trapezoid(A4_r[i],x_range)
    A_r5_trap[i] = trapezoid(A5_r[i],x_range)
    A_r6_trap[i] = trapezoid(A6_r[i],x_range)
        
    A_r[i] = A_r1_trap[i] - A_r2_trap[i] + A_r3_trap[i] - A_r4_trap[i] - A_r5_trap[i] - A_r6_trap[i]
    
    ME_r[i] = A_r_LO[i] + A_r[i]
    
#print(A_r_LO)
#print(A_r1_trap)
#print(A_r2_trap)
#print(A_r3_trap)
#print(A_r4_trap)
#print(A_r5_trap)
#print(A_r6_trap)
#print(A_r)
print(ME_r)

[ 2.27019404  2.2604784   2.09131268  1.7602735   1.30378654  0.7776532
  0.23880252 -0.21494316 -0.64192055]


In [ ]:
# ME imaginary
A_i_LO = np.zeros(len(z))

# 2D array where each individual array is for the different z values. Each input in each array is for a differnt x value
A1_i = np.zeros((len(z),len(x_range)))
A2_i = np.zeros((len(z),len(x_range)))
A3_i = np.zeros((len(z),len(x_range)))
A4_i = np.zeros((len(z),len(x_range)))
A5_i = np.zeros((len(z),len(x_range)))
A6_i = np.zeros((len(z),len(x_range)))

A_i_NLO = np.zeros((len(z),len(x_range)))
A_i1_trap = np.zeros(len(z))
A_i2_trap = np.zeros(len(z))
A_i3_trap = np.zeros(len(z))
A_i4_trap = np.zeros(len(z))
A_i5_trap = np.zeros(len(z))
A_i6_trap = np.zeros(len(z))
A_i = np.zeros(len(z))

ME_i = np.zeros(len(z))

#LO
A_i_LO_func = lambda z: quad(lambda x: np.sin(x*Pz*z) * (N * x**a * (1-x)**b),0,1)[0]

#NLO
# Theta function implemented by changing integration bound to depend on x*(1-delta)
iA1_func = lambda x,z: (1/x)*constant * fixed_quad(lambda y:np.sin(y*Pz*z) * ((2*y/(x-y))*(np.log((x*Pz/mu)**2) + np.log(4*(y/x)*(1-(y/x)))) - (2*y/(x-y))), 0, x*(1-delta), n=100000)[0] * (N * x**a * (1-x)**b)
iA2_func = lambda x,z: np.sin(x*Pz*z) * constant * fixed_quad(lambda xi: ((2*xi/(1-xi))*(np.log((x*Pz/mu)**2) + np.log(4*xi*(1-xi))) - (2*xi/(1-xi))), x, 1-delta, n=100000)[0] * (N * x**a * (1-x)**b)
iA3_func = lambda x,z: (1/x)* fixed_quad(lambda y: np.sin(y*Pz*z)*constant*(2*(y/x)/(1-(y/x)) *np.log((y/x)/((y/x)-1)) + 2/(y/x)), x*(1+delta), upper_bound, n=100000)[0] * (N * x**a * (1-x)**b)
iA4_func = lambda x,z: np.sin(x*Pz*z)*constant*fixed_quad(lambda xi: ((2*xi/(1-xi))*np.log(xi/(xi-1)) + (2/xi)), 1+delta, upper_bound,n=100000)[0]* (N * x**a * (1-x)**b)
iA5_func = lambda x,z: (-1/x)*constant* fixed_quad(lambda y: np.sin(y*Pz*z) * ((2*y/(x+y))*np.log(y/(x+y)) + 2/(1+(y/x))), 0, upper_bound,n=100000)[0] * conf['transversity'].get_C(x, Q2)[1]
iA6_func = lambda x: np.sin(x*Pz*z)*constant * fixed_quad(lambda xi:((-2*xi/(1-xi))*np.log(xi/(xi-1)) + (2/(1-xi))), -1*upper_bound, -x, n=100000)[0] * conf['transversity'].get_C(x, Q2)[1]   

for i in range(0,len(z)):
    A_i_LO[i] = A_i_LO_func(z[i])
    for j in range(0,len(x_range)):
        A1_i[i][j] = np.array(iA1_func(x_range[j], z[i]))
        A2_i[i][j] = np.array(iA2_func(x_range[j], z[i]))
        A3_i[i][j] = np.array(iA3_func(x_range[j], z[i]))
        A4_i[i][j] = np.array(iA4_func(x_range[j], z[i]))
        A5_i[i][j] = np.array(iA5_func(x_range[j], z[i]))
        A6_i[i][j] = np.array(iA6_func(x_range[j], z[i]))
        
    A_i1_trap[i] = trapezoid(A1_i[i],x_range)
    A_i2_trap[i] = trapezoid(A2_i[i],x_range)
    A_i3_trap[i] = trapezoid(A3_i[i],x_range)
    A_i4_trap[i] = trapezoid(A4_i[i],x_range)
    A_i5_trap[i] = trapezoid(A5_i[i],x_range)
    A_i6_trap[i] = trapezoid(A6_i[i],x_range)
        
    A_i[i] = A_i1_trap[i] - A_i2_trap[i] + A_i3_trap[i] - A_i4_trap[i] - A_i5_trap[i] - A_i6_trap[i]
    
    ME_i[i] = A_i_LO[i] + A_i[i]
    
#print(A_i_LO)
#print(A_i1_trap)
#print(A_i2_trap)
#print(A_i3_trap)
#print(A_i4_trap)
#print(A_i5_trap)
#print(A_i6_trap)
#print(A_i)
#print(ME_i)

In [5]:
plt.scatter(z, A_r_LO)
plt.scatter(z, -A_i_LO)
plt.xlabel("z/a")
plt.ylabel("$ME_{LO}$")
plt.show()

NameError: name 'z' is not defined

In [6]:
plt.scatter(z, A_r_NLO_trap)
plt.scatter(z, -A_i_NLO_trap)
plt.xlabel("z/a")
plt.ylabel("$ME_{NLO}$")
plt.show()

NameError: name 'z' is not defined

In [ ]:
plt.scatter(z, ME_r)
plt.scatter(z, -ME_i)
plt.xlabel("z/a")
plt.ylabel("ME")
plt.show()

In [ ]:
plt.scatter(z, ME_r)
plt.scatter(z, -ME_i)
plt.xlabel("z/a")
plt.ylabel("ME")
plt.show()